In [1]:
import numpy as np
import torch

$${tfidf}(w_{i},d,D) = tf(w_{i},d) * idf(w_{i},D)$$
$$ tf(w_{i},d) = \frac{f(w_{i},d)}{max{(f(w_{i},d):w_{j} \in V)}}$$
$$idf(w_{i},D) = log_{10} \frac {|D|}{|d^{'} \in D: w_{i} \in d^{'}|}$$

In [ ]:
def gather_20newsgroups_data():
    path = ''
    dirs = [path+dir_name+'/' 
            for dir_name in listdir(path)
            if not isfile(path + dir_name)]